In [ ]:
import hashlib
import torch

In [ ]:
from pathlib import Path

import numpy as np
from bioimageio.spec.model.v0_5 import (
    ArchitectureFromLibraryDescr,
    ArchitectureFromFileDescr,
    Author,
    CiteEntry,
    AxisBase,
    AxisId,
    BatchAxis,
    ChannelAxis,
    EnvironmentFileDescr,
    FileDescr,
    FixedZeroMeanUnitVarianceDescr,
    FixedZeroMeanUnitVarianceKwargs,
    Identifier,
    InputTensorDescr,
    ModelDescr,
    OutputTensorDescr,
    PytorchStateDictWeightsDescr,
    SpaceInputAxis,
    SpaceOutputAxis,
    TensorId,
    Version,
    WeightsDescr,
)

In [ ]:
# README.md file
doc_md = "doc.md"

In [ ]:
# model input
input_path = "./input_sample.npy"
input_sample = np.load(input_path)
# building axes
in_axes = [BatchAxis()]  # batch is always there!
# channel
in_axes.append(
    ChannelAxis(channel_names=[Identifier("channel")])
)
# spatial dims
in_axes.append(
    SpaceInputAxis(id=AxisId("y"), size=input_sample.shape[2])
)
in_axes.append(
    SpaceInputAxis(id=AxisId("x"), size=input_sample.shape[3])
)
# input descriptor
input_descr = InputTensorDescr(
    id=TensorId("input"),
    axes=in_axes,
    test_tensor=FileDescr(source=input_path),
)


In [ ]:
# model outputs: masks and flows
# masks
output1_path = "./output1_sample_masks.npy"
output1_sample = np.load(output1_path)
# building axes
out1_axes = [BatchAxis()]  # batch is always there!
# spatial dims
out1_axes.append(
    SpaceOutputAxis(id=AxisId("y"), size=output1_sample.shape[1])
)
out1_axes.append(
    SpaceOutputAxis(id=AxisId("x"), size=output1_sample.shape[2])
)
# output descriptor
output1_descr = OutputTensorDescr(
    id=TensorId("masks"),
    axes=out1_axes,
    test_tensor=FileDescr(source=output1_path),
)

# flows
output2_path = "./output2_sample_flows.npy"
output2_sample = np.load(output2_path)
# building axes
out2_axes = [BatchAxis()]  # batch is always there!
# channel
out2_axes.append(
    ChannelAxis(channel_names=[
        Identifier(f"ch_{i}")
        for i in range(output2_sample.shape[1])
    ])
)
# spatial dims
out2_axes.append(
    SpaceOutputAxis(id=AxisId("y"), size=output2_sample.shape[2])
)
out2_axes.append(
    SpaceOutputAxis(id=AxisId("x"), size=output2_sample.shape[3])
)
# output descriptor
output2_descr = OutputTensorDescr(
    id=TensorId("flows"),
    axes=out2_axes,
    test_tensor=FileDescr(source=output2_path),
)


[https://bioimage-io.github.io/spec-bioimage-io/bioimageio/spec/model/v0_5.html#ArchitectureDescr](https://bioimage-io.github.io/spec-bioimage-io/bioimageio/spec/model/v0_5.html#ArchitectureDescr)

In [ ]:
# model arch & weights
model_weights_file = Path("./model_weights.pth")

model_src_file = "./model.py"
with open(model_src_file, "rb") as f:
    model_sha256 = hashlib.sha256(f.read()).hexdigest()
# print(model_sha256)

# params to instantiate the model (pass to __init__)
model_kwargs = {
    "model_type": "cyto",
    "channels": [0, 0],
    "diameter": None,
    "batch_size": 8,
    "do_3D": False
}

arch_descr = ArchitectureFromFileDescr(
    source=model_src_file,
    sha256=model_sha256,
    callable="CellPoseWrapper",
    kwargs=model_kwargs
)

pytorch_version = str(torch.__version__)
env_path = "./env.yml"

weights_descr = WeightsDescr(
    pytorch_state_dict=PytorchStateDictWeightsDescr(
        source=model_weights_file,
        architecture=arch_descr,
        pytorch_version=Version(pytorch_version),
        dependencies=EnvironmentFileDescr(source=env_path),
    ),
)

In [ ]:
# authors
au1 = Author(
    name="Carsen Stringer",
    email=None,
    affiliation="HHMI Janelia Research Campus",
    orcid=None,
    github_user=None
)
au2 = Author(
    name="Tim Wang",
    email=None,
    affiliation=None,
    orcid=None,
    github_user=None
)

au3 = Author(
    name="Michalis Michaelos",
    email=None,
    affiliation=None,
    orcid=None,
    github_user=None
)

au4 = Author(
    name="Marius Pachitariu",
    email=None,
    affiliation=None,
    orcid=None,
    github_user=None
)

authors = [au1, au2, au3, au4]

In [ ]:
citation = CiteEntry(
    text="""Stringer, C., Wang, T., Michaelos, M., & Pachitariu, M. (2021). Cellpose: a generalist algorithm for cellular segmentation. Nature methods, 18(1), 100-106.""",
    doi="10.1038/s41592-020-01018-x"
)

In [ ]:
# model
model_descr = ModelDescr(
    name="CellPose(cyto)",
    authors=authors,
    description="CellPose 'cyto' model",
    documentation=doc_md,
    inputs=[input_descr],
    outputs=[output1_descr, output2_descr],
    tags=["Cell Segmentation", "Segmentation"],
    # covers=["cover1.png"],
    links=[
        "https://github.com/mouseland/cellpose",
    ],
    gitrepo="https://github.com/juglab/cellpose-bmz-wrapper",
    license="BSD-3-Clause",
    version="0.1.0",
    weights=weights_descr,
    cite=[citation],
)

In [ ]:
model_descr.validation_summary.display()

In [ ]:
from bioimageio.core import test_model

summary = test_model(model_descr)
summary.display()

In [ ]:
from bioimageio.spec import save_bioimageio_package

save_bioimageio_package(model_descr, output_path=Path("cellpose.zip"))